In [35]:
import pandas as pd

df_downloaded = pd.read_csv("clean_data2.csv")
display(df_downloaded.head())

,Unnamed: 0,ticket_id,comment,photo,photo_after,address,subdistrict,district,province,timestamp,...,count_reopen,last_activity,latitude,longitude,type 1,type 2,type 3,organization_1,organization_2,organization_3
0,1,2021-CGPMUN,น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,...,0,2022-06-21 08:21:09.532782+00,13.67891,100.66709,น้ำท่วม,ร้องเรียน,NaN,เขตประเวศ,ฝ่ายโยธา เขตประเวศ,NaN
1,2,2021-7XATFA,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,...,0,2022-06-06 01:17:12.272904+00,13.72060,100.52649,สะพาน,NaN,NaN,เขตสาทร,NaN,NaN
2,4,2021-DVEWYM,ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,...,0,2022-08-12 07:18:44.884945+00,13.82280,100.59165,น้ำท่วม,ถนน,NaN,เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว,NaN
3,7,2021-8N9ZP8,คนเอาขยะมาทิ้งจนกลายเป็นกองขยะค่ะ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,110 ซอย มีสุข แขวง หนองบอน เขต ประเวศ กรุงเทพม...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-18 14:50:52.437512+00,...,0,2024-11-26 04:17:39.760344+00,13.67083,100.64690,ความสะอาด,NaN,NaN,เขตประเวศ,ฝ่ายเทศกิจ เขตประเวศ,ฝ่ายรักษาความสะอาดฯ เขตประเวศ
4,9,2021-8BTWZB,ขอแจ้งเรื่องท่อระบายน้ำบนถนนในซอยเสียหาย เป็นร...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,70 ซอย เฉลิมพระเกียรติ ร. 9 แขวง หนองบอน เขต ป...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-22 10:15:33.294829+00,...,0,2022-06-20 13:12:04.99444+00,13.68158,100.65440,ท่อระบายน้ำ,NaN,NaN,เขตประเวศ,ฝ่ายโยธา เขตประเวศ,NaN


In [36]:
display(df_downloaded['timestamp'].min())
display(df_downloaded['timestamp'].max())

'2021-09-19 14:56:08.924992+00'

'2025-01-16 02:53:34.290375+00'

In [37]:
df = df_downloaded.sample(100000)

# scape condo

In [38]:
import requests
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

# ---------------------------------------------------------
# PART 1: Fetch External Data (Condo/Apartment from OSM)
# ---------------------------------------------------------
print("1. Fetching Condo Data from OpenStreetMap API...")

overpass_url = "http://overpass-api.de/api/interpreter"
# Bounding Box ครอบคลุมพื้นที่ กทม. และปริมณฑล
bbox = "(13.45, 100.30, 14.05, 101.00)"

# Query: หาตึกที่เป็นที่พักอาศัยและมีชื่อระบุ
query = f"""
[out:json][timeout:60];
(
  node["residential"="condominium"]{bbox};
  way["residential"="condominium"]{bbox};
  node["building"="apartments"]{bbox};
  way["building"="apartments"]{bbox};
  node["name"~"Condo|Residence|Mansion|Apartment",i]{bbox};
);
out center;
"""

try:
    response = requests.get(overpass_url, params={'data': query})
    data = response.json()

    condo_list = []
    for item in data.get('elements', []):
        # ดึงพิกัด (รองรับทั้ง node และ way)
        lat = item.get('lat') or item.get('center', {}).get('lat')
        lon = item.get('lon') or item.get('center', {}).get('lon')

        # ดึงชื่อ (พยายามเอาภาษาไทยก่อน ถ้าไม่มีเอาอังกฤษ)
        tags = item.get('tags', {})
        name = tags.get('name:th') or tags.get('name:en') or tags.get('name')

        if lat and lon and name:
            condo_list.append({
                'condo_name': name,
                'lat': lat,
                'lon': lon
            })

    df_condo = pd.DataFrame(condo_list)
    print(f"   -> Raw data fetched: {len(df_condo)} records")

except Exception as e:
    print(f"Error fetching data: {e}")
    # ถ้า API พัง ให้สร้าง df_condo เปล่าๆ เพื่อกันโค้ด Error
    df_condo = pd.DataFrame(columns=['condo_name', 'lat', 'lon'])

# ---------------------------------------------------------
# PART 2: Clean External Data
# ---------------------------------------------------------
print("2. Cleaning Condo Data...")

if not df_condo.empty:
    # 2.1 ลบข้อมูลซ้ำ (ชื่อเดียวกันเอาไว้อันเดียว)
    df_condo.drop_duplicates(subset=['condo_name'], keep='first', inplace=True)

    # 2.2 กรอง Noise (ตัดสำนักงานขาย, โชว์รูม, ร้านค้า)
    noise_keywords = ['Sale Gallery', 'Office', 'Showroom', 'สำนักงานขาย', 'ร้าน', 'Shop', '7-Eleven', 'Market']
    pattern = '|'.join(noise_keywords)
    df_condo = df_condo[~df_condo['condo_name'].str.contains(pattern, case=False, na=False)]

    # 2.3 แปลงพิกัดเป็นตัวเลข
    df_condo['lat'] = pd.to_numeric(df_condo['lat'], errors='coerce')
    df_condo['lon'] = pd.to_numeric(df_condo['lon'], errors='coerce')

    print(f"   -> Cleaned data remaining: {len(df_condo)} records")
else:
    print("   -> No condo data available to process.")

# ---------------------------------------------------------
# PART 3: Integrate with Traffy Data & Calculate Features
# ---------------------------------------------------------
print("3. Calculating Distance and Density...")

# ใช้ตัวแปร clean_organization จากขั้นตอนก่อนหน้า
target_df = df.copy()
target_df['province'] = target_df['province'].str.strip()
target_df = target_df[target_df['province'] == 'กรุงเทพมหานคร']

# แปลงพิกัด Traffy เป็นตัวเลข
target_df['latitude'] = pd.to_numeric(target_df['latitude'], errors='coerce')
target_df['longitude'] = pd.to_numeric(target_df['longitude'], errors='coerce')
target_df = target_df.dropna(subset=['latitude', 'longitude'])

if not df_condo.empty:
    # เตรียมพิกัดสำหรับ cKDTree
    condo_coords = df_condo[['lat', 'lon']].values
    traffy_coords = target_df[['latitude', 'longitude']].values

    # สร้าง Tree ครั้งเดียว ใช้ได้ทั้งสองงาน
    tree = cKDTree(condo_coords)

    # --- TASK A: หาคอนโดที่ "ใกล้ที่สุด" (Distance) ---
    # k=1 คือเอาแค่ 1 จุดที่ใกล้ที่สุด
    dist_deg, indices = tree.query(traffy_coords, k=1)

    # แปลงองศาเป็น Km (1 องศา ≈ 111.12 km)
    target_df['dist_to_nearest_condo_km'] = dist_deg * 111.12
    target_df['nearest_condo_name'] = df_condo.iloc[indices]['condo_name'].values

    # --- TASK B: นับจำนวนคอนโดในรัศมี 1 กม. (Density) ---
    # รัศมี 1 km แปลงเป็นองศา
    radius_deg = 1 / 111.12

    # query_ball_point จะคืนค่าเป็น list ของ index ที่อยู่ในวงกลม
    indices_in_radius = tree.query_ball_point(traffy_coords, r=radius_deg)

    # นับจำนวนสมาชิกใน list
    target_df['condo_count_1km'] = [len(x) for x in indices_in_radius]

    print("   -> Calculation Completed!")
else:
    print("   -> Skipping calculation (No condo data).")

1. Fetching Condo Data from OpenStreetMap API...
   -> Raw data fetched: 1164 records
2. Cleaning Condo Data...
   -> Cleaned data remaining: 1041 records
3. Calculating Distance and Density...
   -> Calculation Completed!


# scape สภาพอากาศ

In [ ]:
import requests
import pandas as pd

def get_historical_weather(start_date, end_date, lat=13.7563, lon=100.5018):
    """
    ดึงข้อมูลจาก Open-Meteo (Archive API)
    Default Location: Bangkok (lat=13.7563, lon=100.5018)
    """
    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": "weather_code", # ดึงรหัสสภาพอากาศรายวัน
        "timezone": "Asia/Bangkok"
    }

    response = requests.get(url, params=params)
    data = response.json()

    # สร้าง DataFrame จากข้อมูลดิบ
    df = pd.DataFrame({
        "date": pd.to_datetime(data["daily"]["time"]),
        "weather_code": data["daily"]["weather_code"]
    })

    return df

def map_weather_condition(code):
    """
    แปลง WMO Weather Code เป็น 4 กลุ่มหลัก:
    - ฟ้าใส (default)
    - ฝนตก (เฉพาะปานกลางถึงหนัก)
    - พายุฝน
    - หมอก
    """

    # หมอก
    if code in [45, 48]:
        return "หมอก"

    # ฝนตก (ปานกลางถึงหนักเท่านั้น)
    # 63 = ฝนปานกลาง, 65 = ฝนหนัก
    # 81 = ฝน shower ปานกลาง, 82 = ฝน shower หนัก
    if code in [63, 65, 81, 82]:
        return "ฝนตก"

    # พายุฝนฟ้าคะนอง
    if code in [95, 96, 99]:
        return "พายุฝน"

    # default = ฟ้าใส
    return "ฟ้าใส"

# 1. กำหนดวันและดึงข้อมูล
start = "2021-09-03"
end = "2025-01-16"

# ดึงข้อมูล (ตัวอย่างพิกัด กรุงเทพฯ)
df_weather = get_historical_weather(start, end)

# 2. แปลง Code เป็นข้อความ (Map values)
df_weather['สภาพอากาศ'] = df_weather['weather_code'].apply(map_weather_condition)

# 3. จัด Format ให้เหลือแค่ 1 Column (โดยให้ Date เป็น Index เพื่อความสวยงาม)
final_df = df_weather.set_index('date')[['สภาพอากาศ']]

# แสดงผลลัพธ์
print(f"ดึงข้อมูลวันที่: {start} ถึง {end}")
print("-" * 30)
print(final_df)

# เช็คจำนวนกลุ่มสภาพอากาศที่มี
print("\nสรุปจำนวนประเภทสภาพอากาศที่พบ:")
print(final_df['สภาพอากาศ'].value_counts())

ดึงข้อมูลวันที่: 2021-09-03 ถึง 2025-01-16
------------------------------
           สภาพอากาศ
date                
2021-09-03     ฟ้าใส
2021-09-04     ฟ้าใส
2021-09-05      ฝนตก
2021-09-06     ฟ้าใส
2021-09-07      ฝนตก
...              ...
2025-01-12     ฟ้าใส
2025-01-13     ฟ้าใส
2025-01-14     ฟ้าใส
2025-01-15     ฟ้าใส
2025-01-16     ฟ้าใส

[1232 rows x 1 columns]

สรุปจำนวนประเภทสภาพอากาศที่พบ:
สภาพอากาศ
ฟ้าใส    850
ฝนตก     382
Name: count, dtype: int64


In [40]:
final_df['สภาพอากาศ'].head()

date
2021-09-03    ฟ้าใส
2021-09-04    ฟ้าใส
2021-09-05     ฝนตก
2021-09-06    ฟ้าใส
2021-09-07     ฝนตก
Name: สภาพอากาศ, dtype: object

In [41]:
min_timestamp = target_df['timestamp'].min()
max_timestamp = target_df['timestamp'].max()

print(f"Minimum Timestamp: {min_timestamp}")
print(f"Maximum Timestamp: {max_timestamp}")

Minimum Timestamp: 2022-02-07 00:15:45.797321+00
Maximum Timestamp: 2025-01-16 02:51:28.8957+00


In [42]:
target_df['timestamp'] = pd.to_datetime(target_df['timestamp'], errors='coerce')
target_df['date'] = target_df['timestamp'].dt.normalize().dt.tz_localize(None)
target_df = pd.merge(target_df, final_df, left_on='date', right_index=True, how='left')

print(target_df[['timestamp', 'date', 'สภาพอากาศ']].head())

                              timestamp       date สภาพอากาศ
30048  2022-06-21 15:15:11.655621+00:00 2022-06-21      ฝนตก
561256 2024-08-24 09:22:14.664393+00:00 2024-08-24     ฟ้าใส
653679 2024-12-22 09:40:35.785924+00:00 2024-12-22     ฟ้าใส
205150 2023-05-08 06:30:10.149692+00:00 2023-05-08      ฝนตก
249164 2023-07-05 16:14:24.792416+00:00 2023-07-05     ฟ้าใส


In [43]:
target_df.head()

,Unnamed: 0,ticket_id,comment,photo,photo_after,address,subdistrict,district,province,timestamp,...,type 2,type 3,organization_1,organization_2,organization_3,dist_to_nearest_condo_km,nearest_condo_name,condo_count_1km,date,สภาพอากาศ
30048,42435,2022-KF4BTR,ขอร่วมบอกความฝันที่อยากให้เมืองดีขึ้น หรือแจ้ง...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,2387 ถนน ลาดพร้าว แขวง คลองจั่น เขตบางกะปิ กรุ...,คลองจั่น,บางกะปิ,กรุงเทพมหานคร,2022-06-21 15:15:11.655621+00:00,...,คลอง,ความปลอดภัย,เขตบางกะปิ,สำนักการระบายน้ำ กทม.,ฝ่ายโยธา เขตบางกะปิ,1.580111,101 Mansion,0,2022-06-21,ฝนตก
561256,663104,2024-EFKAYD,หน้า บ้านเลขที่ 87/67 \nเปลี่ยนฝาท่อได้มั๊ยค่ะ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,87/90 ถ. สุขสวัสดิ์ แขวงจอมทอง เขตจอมทอง กรุงเ...,จอมทอง,จอมทอง,กรุงเทพมหานคร,2024-08-24 09:22:14.664393+00:00,...,ท่อระบายน้ำ,NaN,เขตจอมทอง,ฝ่ายโยธา เขตจอมทอง,NaN,1.961611,Saransook Supreme,0,2024-08-24,ฟ้าใส
653679,767554,2024-6VRBL8,ป้ายโฆษณาติดไม่ได้รับอนุญาต,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,4/1 ซอย กรุงธนบุรี 5 แขวงคลองต้นไทร เขตคลองสาน...,คลองต้นไทร,คลองสาน,กรุงเทพมหานคร,2024-12-22 09:40:35.785924+00:00,...,NaN,NaN,เขตคลองสาน,ฝ่ายเทศกิจ เขตคลองสาน,NaN,0.215801,เดอะ แบงค็อค สาทร–ตากสิน,15,2024-12-22,ฟ้าใส
205150,267408,J8NU3B,ปัญหา: เหตุเดือดร้อนรำคาญ เนื่องจาก ปัญหาได้รั...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,เทศบาลรังสรรเหนือ ซอย4 เทศบาลรังสรรเหนือ ลาดยา...,ลาดยาว,จตุจักร,กรุงเทพมหานคร,2023-05-08 06:30:10.149692+00:00,...,สอบถาม,ร้องเรียน,ฝ่ายสิ่งแวดล้อม แขวงลาดยาว จตุจักร,เขตจตุจักร,ฝ่ายสิ่งแวดล้อมฯ เขตจตุจักร,0.593727,Condo.Bahn Prachanives1,1,2023-05-08,ฝนตก
249164,315870,2023-GK3WRN,ไฟถนนนดับนานแล้ว,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,64 ถนน กล้วยน้ำไท แขวงพระโขนง เขตคลองเตย กรุงเ...,พระโขนง,คลองเตย,กรุงเทพมหานคร,2023-07-05 16:14:24.792416+00:00,...,NaN,NaN,เขตคลองเตย,สำนักการโยธา กทม.,สำนักงานก่อสร้างและบูรณะ สำนักการโยธา (สกบ.สนย...,0.062582,aspire rama 4,14,2023-07-05,ฟ้าใส


# scape ความหนาแน่นคน

In [44]:
# ==============================================================================
# DATA PIPELINE (FIXED): Scrape Bangkok Population with Headers
# ==============================================================================
import pandas as pd
import requests

# 1. Scrape Data from Wikipedia (with Headers)
print("1. Scraping population data from Wikipedia...")
url = "https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok"

# [FIX] เพิ่ม User-Agent เพื่อไม่ให้โดนบล็อก 403
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

try:
    # ยิง Request แบบมี Header
    response = requests.get(url, headers=headers)
    response.raise_for_status() # เช็คว่าผ่านไหม (200 OK)

    # อ่านตารางจากเนื้อหา HTML ที่ได้มา
    tables = pd.read_html(response.text)

    # ตารางแรกคือตารางเขต
    df_pop = tables[0]

    # Rename Columns
    # (ชื่อคอลัมน์ Wiki บางทีมี [1] ติดมา ต้องใช้ .str.contains หรือ rename แบบกว้างๆ)
    print("   -> Raw columns:", df_pop.columns.tolist())

    # Map ชื่อคอลัมน์ใหม่ (สังเกตตำแหน่งคอลัมน์จากหน้าเว็บ)
    # ปกติ: District (Khet) | Thai | Population | ...
    df_pop = df_pop.rename(columns={
        df_pop.columns[0]: 'district_en', # District (Khet)
        df_pop.columns[1]: 'district',    # Thai name
        df_pop.columns[2]: 'population',  # Population
        df_pop.columns[4]: 'area_sqkm'    # Area
    })

    # Clean Data
    # ลบลูกน้ำ, ลบ footnote [1], แปลงเป็นตัวเลข
    df_pop['population'] = df_pop['population'].astype(str).str.replace(r'\[.*\]', '', regex=True).str.replace(',', '')
    df_pop['area_sqkm'] = df_pop['area_sqkm'].astype(str).str.replace(r'\[.*\]', '', regex=True).str.replace(',', '')

    df_pop['population'] = pd.to_numeric(df_pop['population'], errors='coerce')
    df_pop['area_sqkm'] = pd.to_numeric(df_pop['area_sqkm'], errors='coerce')

    # สร้าง Feature: ความหนาแน่นประชากร (คน/ตร.กม.)
    df_pop['pop_density'] = df_pop['population'] / df_pop['area_sqkm']

    print(f"   -> Scraped {len(df_pop)} districts successfully.")
    display(df_pop[['district', 'population', 'pop_density']].head())

except Exception as e:
    print(f"❌ Error scraping: {e}")
    df_pop = pd.DataFrame()

# ---------------------------------------------------------
# 2. Merge with target_df
# ---------------------------------------------------------
if not df_pop.empty and 'target_df' in globals():
    print("\n2. Merging Population Data to Main Dataset...")

    # Clean ชื่อเขตใน df_pop ให้ตรงกับ target_df (ลบคำว่า "เขต")
    # ตัวอย่าง: "เขตพระนคร" -> "พระนคร"
    df_pop['district'] = df_pop['district'].astype(str).str.replace('เขต', '').str.strip()

    # Merge (Left Join)
    # ก่อน Merge ลบคอลัมน์เก่าออกก่อนถ้ามี (กันซ้ำ)
    if 'pop_density' in target_df.columns:
        target_df = target_df.drop(columns=['pop_density', 'population'])

    target_df = target_df.merge(df_pop[['district', 'population', 'pop_density']],
                                on='district',
                                how='left')

    # Fill Missing (สำหรับแถวที่ Join ไม่ติด)
    mean_density = target_df['pop_density'].mean()
    target_df['pop_density'] = target_df['pop_density'].fillna(mean_density)

    print("✅ Merge Completed!")
    print(target_df[['ticket_id', 'district', 'pop_density']].head())

else:
    print("❌ Cannot merge: Data not ready or 'target_df' missing.")

1. Scraping population data from Wikipedia...
❌ Error scraping: Missing optional dependency 'lxml'.  Use pip or conda to install lxml.
❌ Cannot merge: Data not ready or 'target_df' missing.


/var/folders/kd/gnv0klmj7w7cs5kxqx4fpkbw0000gn/T/ipykernel_4256/2966272893.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


In [45]:
target_df.head()

,Unnamed: 0,ticket_id,comment,photo,photo_after,address,subdistrict,district,province,timestamp,...,type 2,type 3,organization_1,organization_2,organization_3,dist_to_nearest_condo_km,nearest_condo_name,condo_count_1km,date,สภาพอากาศ
30048,42435,2022-KF4BTR,ขอร่วมบอกความฝันที่อยากให้เมืองดีขึ้น หรือแจ้ง...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,2387 ถนน ลาดพร้าว แขวง คลองจั่น เขตบางกะปิ กรุ...,คลองจั่น,บางกะปิ,กรุงเทพมหานคร,2022-06-21 15:15:11.655621+00:00,...,คลอง,ความปลอดภัย,เขตบางกะปิ,สำนักการระบายน้ำ กทม.,ฝ่ายโยธา เขตบางกะปิ,1.580111,101 Mansion,0,2022-06-21,ฝนตก
561256,663104,2024-EFKAYD,หน้า บ้านเลขที่ 87/67 \nเปลี่ยนฝาท่อได้มั๊ยค่ะ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,87/90 ถ. สุขสวัสดิ์ แขวงจอมทอง เขตจอมทอง กรุงเ...,จอมทอง,จอมทอง,กรุงเทพมหานคร,2024-08-24 09:22:14.664393+00:00,...,ท่อระบายน้ำ,NaN,เขตจอมทอง,ฝ่ายโยธา เขตจอมทอง,NaN,1.961611,Saransook Supreme,0,2024-08-24,ฟ้าใส
653679,767554,2024-6VRBL8,ป้ายโฆษณาติดไม่ได้รับอนุญาต,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,4/1 ซอย กรุงธนบุรี 5 แขวงคลองต้นไทร เขตคลองสาน...,คลองต้นไทร,คลองสาน,กรุงเทพมหานคร,2024-12-22 09:40:35.785924+00:00,...,NaN,NaN,เขตคลองสาน,ฝ่ายเทศกิจ เขตคลองสาน,NaN,0.215801,เดอะ แบงค็อค สาทร–ตากสิน,15,2024-12-22,ฟ้าใส
205150,267408,J8NU3B,ปัญหา: เหตุเดือดร้อนรำคาญ เนื่องจาก ปัญหาได้รั...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,เทศบาลรังสรรเหนือ ซอย4 เทศบาลรังสรรเหนือ ลาดยา...,ลาดยาว,จตุจักร,กรุงเทพมหานคร,2023-05-08 06:30:10.149692+00:00,...,สอบถาม,ร้องเรียน,ฝ่ายสิ่งแวดล้อม แขวงลาดยาว จตุจักร,เขตจตุจักร,ฝ่ายสิ่งแวดล้อมฯ เขตจตุจักร,0.593727,Condo.Bahn Prachanives1,1,2023-05-08,ฝนตก
249164,315870,2023-GK3WRN,ไฟถนนนดับนานแล้ว,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,64 ถนน กล้วยน้ำไท แขวงพระโขนง เขตคลองเตย กรุงเ...,พระโขนง,คลองเตย,กรุงเทพมหานคร,2023-07-05 16:14:24.792416+00:00,...,NaN,NaN,เขตคลองเตย,สำนักการโยธา กทม.,สำนักงานก่อสร้างและบูรณะ สำนักการโยธา (สกบ.สนย...,0.062582,aspire rama 4,14,2023-07-05,ฟ้าใส


# scape ราคา condo/ตรม

In [46]:
import requests
import pandas as pd
import json
from typing import List, Dict
import time
import numpy as np
import cloudscraper
import random

# Set up headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
}

# Mapping of district codes to district names
district_mapping = {
    'TH1050': 'บางบอน',
    'TH1006': 'บางกะปิ',
    'TH1040': 'บางแค',
    'TH1005': 'บางเขน',
    'TH1031': 'บางคอแหลม',
    'TH1021': 'บางขุนเทียน',
    'TH1047': 'บางนา',
    'TH1025': 'บางพลัด',
    'TH1004': 'บางรัก',
    'TH1029': 'บางซื่อ',
    'TH1020': 'บางกอกน้อย',
    'TH1016': 'บางกอกใหญ่',
    'TH1027': 'บึงกุ่ม',
    'TH1030': 'จตุจักร',
    'TH1035': 'จอมทอง',
    'TH1026': 'ดินแดง',
    'TH1036': 'ดอนเมือง',
    'TH1002': 'ดุสิต',
    'TH1017': 'ห้วยขวาง',
    'TH1043': 'คันนายาว',
    'TH1046': 'คลองสามวา',
    'TH1018': 'คลองสาน',
    'TH1033': 'คลองเตย',
    'TH1041': 'หลักสี่',
    'TH1011': 'ลาดกระบัง',
    'TH1038': 'ลาดพร้าว',
    'TH1010': 'มีนบุรี',
    'TH1003': 'หนองจอก',
    'TH1023': 'หนองแขม',
    'TH1007': 'ปทุมวัน',
    'TH1022': 'ภาษีเจริญ',
    'TH1014': 'พญาไท',
    'TH1009': 'พระโขนง',
    'TH1001': 'พระนคร',
    'TH1008': 'ป้อมปราบศัตรูพ่าย',
    'TH1032': 'ประเวศ',
    'TH1024': 'ราษฏร์บูรณะ',
    'TH1037': 'ราชเทวี',
    'TH1042': 'สายไหม',
    'TH1013': 'สัมพันธวงศ์',
    'TH1044': 'สะพานสูง',
    'TH1028': 'สาทร',
    'TH1034': 'สวนหลวง',
    'TH1019': 'ตลิ่งชัน',
    'TH1048': 'ทวีวัฒนา',
    'TH1015': 'ธนบุรี',
    'TH1049': 'ทุ่งครุ',
    'TH1045': 'วังทองหลาง',
    'TH1039': 'วัฒนา',
    'TH1012': 'ยานนาวา',
}

import cloudscraper

def fetch_ddproperty_all_districts(max_pages_per_district: int = 2) -> pd.DataFrame:
    all_listings = []
    base_url = "https://www.ddproperty.com/_next/data/Ea7XSXCxje9p33n-MOlyw/%E0%B8%A3%E0%B8%A7%E0%B8%A1%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%81%E0%B8%B2%E0%B8%A8%E0%B8%82%E0%B8%B2%E0%B8%A2.json"
    
    # สร้าง scraper ที่หลบ Cloudflare ได้
    scraper = cloudscraper.create_scraper(
        browser={
            'browser': 'chrome',
            'platform': 'windows',
            'mobile': False
        }
    )
    
    district_codes = list(district_mapping.keys())
    total_districts = len(district_codes)
    
    for district_idx, district_code in enumerate(district_codes, 1):
        district_name = district_mapping[district_code]
        print(f"\n[{district_idx}/{total_districts}] Processing: {district_code} ({district_name})")
        
        for page in range(1, max_pages_per_district + 1):
            try:
                params = {
                    'page': page,
                    'districtCode': district_code,
                    'isCommercial': False,
                    'listingType': 'sale'
                }
                
                print(f"  Fetching page {page}...", end=' ')
                response = scraper.get(base_url, params=params, timeout=15)
                response.raise_for_status()
                
                data = response.json()
                
                # ส่วนที่เหลือเหมือนเดิม...
                page_data = data['pageProps'].get('pageData', {})
                listings_data = page_data.get('data', {})
                listings = listings_data.get('listingsData', [])
                
                print(f"({len(listings)} listings)")
                
                for listing in listings:
                    listing_data = listing.get('listingData', {})
                    
                    price_per_area_str = listing_data.get('pricePerArea', {}).get('localeStringValue', '')
                    price_per_sqm = None
                    if price_per_area_str:
                        try:
                            price_per_sqm = float(price_per_area_str.split('฿')[1].split('/')[0].replace(',', '').strip())
                        except:
                            pass
                    
                    area_sqm = None
                    media_carousel = listing_data.get('mediaCarousel', {})
                    if isinstance(media_carousel, dict):
                        carousel_items = media_carousel.get('items', [])
                        for item in carousel_items:
                            if isinstance(item, dict) and 'label' in item:
                                label = item.get('label', '')
                                if 'ตร.ม' in label or 'ตารางเมตร' in label:
                                    try:
                                        area_sqm = float(label.split()[0].replace(',', ''))
                                    except:
                                        pass
                    
                    if price_per_sqm is None and listing_data.get('price', {}).get('value') and area_sqm:
                        price_per_sqm = listing_data.get('price', {}).get('value') / area_sqm
                    
                    item = {
                        'ชื่อ condo': listing_data.get('localizedTitle'),
                        'latitude': np.nan,
                        'longitude': np.nan,
                        'ราคา': listing_data.get('price', {}).get('value'),
                        'ตารางเมตร': area_sqm,
                        'ราคาต่อตารางเมตร': price_per_sqm,
                        'เขต': district_code,
                        'ชื่อเขต': district_name,
                    }
                    
                    all_listings.append(item)
                
                time.sleep(random.uniform(2, 4))  # เพิ่ม random delay
                
            except Exception as e:
                print(f"  Error: {e}")
                time.sleep(5)
                continue
    
    df = pd.DataFrame(all_listings)
    return df

# Fetch the data
print("Starting to scrape DDProperty condo data from all Bangkok districts...")
print("This may take a few minutes...")
condo_df = fetch_ddproperty_all_districts(max_pages_per_district=2)

print(f"\n{'='*60}")
print(f"SUMMARY")
print(f"{'='*60}")
print(f"Total listings scraped: {len(condo_df)}")
print(f"\nDataFrame shape: {condo_df.shape}")
print(f"\nDataFrame columns: {list(condo_df.columns)}")

print(f"\n{'='*60}")
print("First 10 rows:")
print(condo_df.head(10))

print(f"\n{'='*60}")
print("Data types:")
print(condo_df.dtypes)

print(f"\n{'='*60}")
print("Statistics for numeric columns:")
print(condo_df[['ราคา', 'ตารางเมตร', 'ราคาต่อตารางเมตร']].describe())

print(f"\n{'='*60}")
print("Listings count by district:")
print(condo_df['ชื่อเขต'].value_counts().sort_index())

Starting to scrape DDProperty condo data from all Bangkok districts...
This may take a few minutes...

[1/50] Processing: TH1050 (บางบอน)
  Fetching page 1... (20 listings)
  Fetching page 2... (20 listings)

[2/50] Processing: TH1006 (บางกะปิ)
  Fetching page 1... (25 listings)
  Fetching page 2... (20 listings)

[3/50] Processing: TH1040 (บางแค)
  Fetching page 1... (20 listings)
  Fetching page 2... (20 listings)

[4/50] Processing: TH1005 (บางเขน)
  Fetching page 1... (24 listings)
  Fetching page 2... (20 listings)

[5/50] Processing: TH1031 (บางคอแหลม)
  Fetching page 1... (22 listings)
  Fetching page 2... (20 listings)

[6/50] Processing: TH1021 (บางขุนเทียน)
  Fetching page 1... (20 listings)
  Fetching page 2... (20 listings)

[7/50] Processing: TH1047 (บางนา)
  Fetching page 1... (22 listings)
  Fetching page 2... (20 listings)

[8/50] Processing: TH1025 (บางพลัด)
  Fetching page 1... (22 listings)
  Fetching page 2... (20 listings)

[9/50] Processing: TH1004 (บางรัก)
  Fetc

In [47]:
# Ensure 'ราคาต่อตารางเมตร' is numeric and coerce errors to NaN
condo_df['ราคาต่อตารางเมตร'] = pd.to_numeric(condo_df['ราคาต่อตารางเมตร'], errors='coerce')

# Filter out rows where 'ราคาต่อตารางเมตร' is NaN
df_condo_price_cleaned = condo_df.dropna(subset=['ราคาต่อตารางเมตร'])

# Group by 'เขต' (code) and 'ชื่อเขต' (name) and calculate the mean of 'ราคาต่อตารางเมตร'
df_avg_condo_price_per_district = df_condo_price_cleaned.groupby(['เขต', 'ชื่อเขต'])['ราคาต่อตารางเมตร'].mean().reset_index()

# Rename the column for clarity
df_avg_condo_price_per_district.rename(columns={'ราคาต่อตารางเมตร': 'avg_price_per_sqm'}, inplace=True)

print("Average condo price per square meter by district:")
display(df_avg_condo_price_per_district.head())

Average condo price per square meter by district:


,เขต,ชื่อเขต,avg_price_per_sqm
0,TH1001,พระนคร,105589.157895
1,TH1002,ดุสิต,149026.350000
2,TH1003,หนองจอก,21087.433333
3,TH1004,บางรัก,246883.239130
4,TH1005,บางเขน,69466.333333


In [48]:
df_avg_condo_price_per_district.sort_values(by='avg_price_per_sqm', ascending=True)

,เขต,ชื่อเขต,avg_price_per_sqm
2,TH1003,หนองจอก,21087.433333
22,TH1023,หนองแขม,28575.939394
9,TH1010,มีนบุรี,32655.625000
49,TH1050,บางบอน,34260.742857
48,TH1049,ทุ่งครุ,35437.567568
20,TH1021,บางขุนเทียน,40885.921053
39,TH1040,บางแค,43162.405405
35,TH1036,ดอนเมือง,49765.605263
34,TH1035,จอมทอง,55027.250000
45,TH1046,คลองสามวา,62589.390244


In [49]:
# Merge df_avg_condo_price_per_district into target_df
# Ensure column names are consistent for merging
target_df = pd.merge(target_df,
                     df_avg_condo_price_per_district[['ชื่อเขต', 'avg_price_per_sqm']],
                     left_on='district',
                     right_on='ชื่อเขต',
                     how='left')

# Drop the redundant 'ชื่อเขต' column from the merge
target_df.drop(columns=['ชื่อเขต'], inplace=True)

# Fill any NaN values in 'avg_price_per_sqm' with the mean, or 0 if a mean is not appropriate
# For now, let's fill with the overall mean or 0 if mean is NaN
if 'avg_price_per_sqm' in target_df.columns:
    mean_price = target_df['avg_price_per_sqm'].mean()
    target_df['avg_price_per_sqm'].fillna(mean_price if pd.notna(mean_price) else 0, inplace=True)

print("First 5 rows of target_df with new 'avg_price_per_sqm' column:")
display(target_df[['district', 'avg_price_per_sqm']].head())

First 5 rows of target_df with new 'avg_price_per_sqm' column:


/var/folders/kd/gnv0klmj7w7cs5kxqx4fpkbw0000gn/T/ipykernel_4256/2651763244.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target_df['avg_price_per_sqm'].fillna(mean_price if pd.notna(mean_price) else 0, inplace=True)


,district,avg_price_per_sqm
0,บางกะปิ,102527.209302
1,จอมทอง,55027.250000
2,คลองสาน,186111.681818
3,จตุจักร,133936.813953
4,คลองเตย,190411.777778


In [50]:
target_df.head()

,Unnamed: 0,ticket_id,comment,photo,photo_after,address,subdistrict,district,province,timestamp,...,type 3,organization_1,organization_2,organization_3,dist_to_nearest_condo_km,nearest_condo_name,condo_count_1km,date,สภาพอากาศ,avg_price_per_sqm
0,42435,2022-KF4BTR,ขอร่วมบอกความฝันที่อยากให้เมืองดีขึ้น หรือแจ้ง...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,2387 ถนน ลาดพร้าว แขวง คลองจั่น เขตบางกะปิ กรุ...,คลองจั่น,บางกะปิ,กรุงเทพมหานคร,2022-06-21 15:15:11.655621+00:00,...,ความปลอดภัย,เขตบางกะปิ,สำนักการระบายน้ำ กทม.,ฝ่ายโยธา เขตบางกะปิ,1.580111,101 Mansion,0,2022-06-21,ฝนตก,102527.209302
1,663104,2024-EFKAYD,หน้า บ้านเลขที่ 87/67 \nเปลี่ยนฝาท่อได้มั๊ยค่ะ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,87/90 ถ. สุขสวัสดิ์ แขวงจอมทอง เขตจอมทอง กรุงเ...,จอมทอง,จอมทอง,กรุงเทพมหานคร,2024-08-24 09:22:14.664393+00:00,...,NaN,เขตจอมทอง,ฝ่ายโยธา เขตจอมทอง,NaN,1.961611,Saransook Supreme,0,2024-08-24,ฟ้าใส,55027.250000
2,767554,2024-6VRBL8,ป้ายโฆษณาติดไม่ได้รับอนุญาต,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,4/1 ซอย กรุงธนบุรี 5 แขวงคลองต้นไทร เขตคลองสาน...,คลองต้นไทร,คลองสาน,กรุงเทพมหานคร,2024-12-22 09:40:35.785924+00:00,...,NaN,เขตคลองสาน,ฝ่ายเทศกิจ เขตคลองสาน,NaN,0.215801,เดอะ แบงค็อค สาทร–ตากสิน,15,2024-12-22,ฟ้าใส,186111.681818
3,267408,J8NU3B,ปัญหา: เหตุเดือดร้อนรำคาญ เนื่องจาก ปัญหาได้รั...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,เทศบาลรังสรรเหนือ ซอย4 เทศบาลรังสรรเหนือ ลาดยา...,ลาดยาว,จตุจักร,กรุงเทพมหานคร,2023-05-08 06:30:10.149692+00:00,...,ร้องเรียน,ฝ่ายสิ่งแวดล้อม แขวงลาดยาว จตุจักร,เขตจตุจักร,ฝ่ายสิ่งแวดล้อมฯ เขตจตุจักร,0.593727,Condo.Bahn Prachanives1,1,2023-05-08,ฝนตก,133936.813953
4,315870,2023-GK3WRN,ไฟถนนนดับนานแล้ว,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,64 ถนน กล้วยน้ำไท แขวงพระโขนง เขตคลองเตย กรุงเ...,พระโขนง,คลองเตย,กรุงเทพมหานคร,2023-07-05 16:14:24.792416+00:00,...,NaN,เขตคลองเตย,สำนักการโยธา กทม.,สำนักงานก่อสร้างและบูรณะ สำนักการโยธา (สกบ.สนย...,0.062582,aspire rama 4,14,2023-07-05,ฟ้าใส,190411.777778


In [51]:
target_df.to_csv('scrape.csv', index=True, encoding='utf-8')